### 1. Data Loading

In [56]:
# Import dependencies
import pandas as pd

In [125]:
# Load the data
df = pd.read_csv("Resources/retail_store_data_large.csv")

# Display the data
display(df.head())

,Date,Store,Region,Product Category,Product Subcategory,Units Sold,Unit Price,Marketing Spend,Revenue,COGS,Gross Profit,Net Profit
0,2022-01-01,Store B,West,Sports,Outdoor,36,434.16,183.14,15629.76,6911.575877,8718.184123,8535.044123
1,2022-01-01,Store F,West,Beauty,Haircare,19,276.49,397.66,5253.31,2320.134775,2933.175225,2535.515225
2,2022-01-01,Store C,South,Grocery,Snacks,61,861.96,314.91,52579.56,21946.025852,30633.534148,30318.624148
3,2022-01-01,Store J,South,Grocery,Fresh Produce,94,372.56,9.22,35020.64,23996.010323,11024.629677,11015.409677
4,2022-01-01,Store B,North,Grocery,Snacks,15,547.82,230.34,8217.30,5510.707350,2706.592650,2476.252650


### 2. Data Cleaning

In [128]:
# 1. Initial inspection
print("Initial shape:", df.shape)
print("\nData types:\n", df.dtypes)
print("\nMissing values:\n", df.isnull().sum())

Initial shape: (100000, 12)

Data types:
 Date                    object
Store                   object
Region                  object
Product Category        object
Product Subcategory     object
Units Sold               int64
Unit Price             float64
Marketing Spend        float64
Revenue                float64
COGS                   float64
Gross Profit           float64
Net Profit             float64
dtype: object

Missing values:
 Date                   0
Store                  0
Region                 0
Product Category       0
Product Subcategory    0
Units Sold             0
Unit Price             0
Marketing Spend        0
Revenue                0
COGS                   0
Gross Profit           0
Net Profit             0
dtype: int64


In [130]:
# 2. Handle duplicates
df = df.drop_duplicates()
print("\nShape after removing duplicates:", df.shape)


Shape after removing duplicates: (100000, 12)


In [132]:
# 3. Fix date format with error handling
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # converts invalid dates to NaT
invalid_dates = df[df['Date'].isna()]
if not invalid_dates.empty:
    print(f"\nWarning: {len(invalid_dates)} rows with invalid dates")

In [107]:
# 4. Standardize text fields
df['Region'] = df['Region'].str.title().str.strip()  # also removes extra whitespace
df['Product Category'] = df['Product Category'].str.title().str.strip()

In [134]:
# 5. Handle missing values more robustly
# For numerical columns, consider median instead of 0 for marketing spend
median_marketing = df['Marketing Spend'].median()
df['Marketing Spend'] = df['Marketing Spend'].fillna(median_marketing)

# For categorical columns
df['Region'] = df['Region'].fillna('Unknown')

In [136]:
# 6. Validate numerical ranges
print("\nValue ranges:")
print("Revenue:", df['Revenue'].min(), "-", df['Revenue'].max())
print("Gross Profit:", df['Gross Profit'].min(), "-", df['Gross Profit'].max())


Value ranges:
Revenue: 6.04 - 98974.26
Gross Profit: 2.419622333484721 - 58681.29466251924


In [138]:
# 7. Calculate derived metrics with safety checks
df['Gross Margin'] = (df['Gross Profit'] / df['Revenue'].replace(0, pd.NA)).round(4)
invalid_margins = df[df['Gross Margin'].isna()]
if not invalid_margins.empty:
    print(f"\nWarning: {len(invalid_margins)} rows with zero revenue")

In [140]:
# 8. Additional quality checks
# Check for negative values where they shouldn't exist
negative_revenue = df[df['Revenue'] < 0]
if not negative_revenue.empty:
    print(f"\nWarning: {len(negative_revenue)} rows with negative revenue")

In [142]:
# 9. Final inspection
print("\nFinal shape:", df.shape)
print("\nSample of cleaned data:")
print(df.sample(5))


Final shape: (100000, 13)

Sample of cleaned data:
            Date    Store Region Product Category Product Subcategory  \
8693  2022-04-06  Store D  North             Toys      Action Figures   
52304 2023-07-28  Store G  North      Electronics              Phones   
65296 2023-12-16  Store A  North          Grocery              Snacks   
87358 2024-08-15  Store B  South             Home             Kitchen   
39228 2023-03-05  Store B   East         Clothing               Women   

       Units Sold  Unit Price  Marketing Spend   Revenue          COGS  \
8693           56      668.86           388.43  37456.16  15961.296607   
52304          67      371.16            30.02  24867.72  13080.861873   
65296           7      243.95           414.39   1707.65    879.211937   
87358          20      166.82           110.81   3336.40   2262.268158   
39228          96        3.34           138.69    320.64    172.513221   

       Gross Profit    Net Profit  Gross Margin  
8693   21494.8

In [144]:
# 10. Export cleaned data with timestamp
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
df.to_csv(f"retail_data_cleaned_{timestamp}.csv", index=False)
df.to_parquet(f"retail_data_cleaned_{timestamp}.parquet")

print("\nCleaning complete! Files saved with timestamp.")


Cleaning complete! Files saved with timestamp.


In [146]:
# Final export from Jupyter (best format for Postgres)
df.to_csv('retail_cleaned_final.csv', index=False, encoding='utf-8')
print("\nCleaning complete! Files .csv saved.")


Cleaning complete! Files .csv saved.


In [148]:
df

,Date,Store,Region,Product Category,Product Subcategory,Units Sold,Unit Price,Marketing Spend,Revenue,COGS,Gross Profit,Net Profit,Gross Margin
0,2022-01-01,Store B,West,Sports,Outdoor,36,434.16,183.14,15629.76,6911.575877,8718.184123,8535.044123,0.5578
1,2022-01-01,Store F,West,Beauty,Haircare,19,276.49,397.66,5253.31,2320.134775,2933.175225,2535.515225,0.5583
2,2022-01-01,Store C,South,Grocery,Snacks,61,861.96,314.91,52579.56,21946.025852,30633.534148,30318.624148,0.5826
3,2022-01-01,Store J,South,Grocery,Fresh Produce,94,372.56,9.22,35020.64,23996.010323,11024.629677,11015.409677,0.3148
4,2022-01-01,Store B,North,Grocery,Snacks,15,547.82,230.34,8217.30,5510.707350,2706.592650,2476.252650,0.3294
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2024-12-31,Store D,North,Electronics,Phones,60,356.09,119.52,21365.40,12248.811158,9116.588842,8997.068842,0.4267
99996,2024-12-31,Store H,North,Toys,Action Figures,50,698.40,156.67,34920.00,14746.134064,20173.865936,20017.195936,0.5777
99997,2024-12-31,Store E,East,Sports,Fitness,66,99.78,42.40,6585.48,4297.967884,2287.512116,2245.112116,0.3474
99998,2024-12-31,Store E,North,Electronics,Phones,95,909.42,73.86,86394.90,38351.228495,48043.671505,47969.811505,0.5561
